<a href="https://colab.research.google.com/github/bannis14/Portfolio-Projects/blob/main/Time_Series_Forecast/Sarimax_Parameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SARIMAX Parameter Tuning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pmdarima

In [ ]:
# change directory
%cd /content/drive/MyDrive/Time Series Forecasting Product

/content/drive/MyDrive/Time Series Forecasting Product


## Libraries and Data

In [ ]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pmdarima as pm
from sklearn.model_selection import ParameterGrid
from pmdarima import model_selection
from statsmodels.tsa.stattools import adfuller

In [ ]:
# Load the data
# YYYY-MM-DD
df = pd.read_csv('nyc_data.csv', index_col=0, parse_dates=True)
df.head()

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077


In [ ]:
# Rename variable
df = df.rename(columns={'Demand': 'y'})
df.head(0)

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,


In [ ]:
# Extract Regressor
X = df.iloc[:, 1:]
X.head(0)

,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,


## Stationarity

*     if dickey-fuller is less than 0.5 it is stationary


In [ ]:
pvalue = adfuller(x=df.y)[1]

In [ ]:
# condition to read test
if pvalue < 0.05:
    print(f"Time series is stationary with a p value of {pvalue}")
else:
    print(f"Time series is not stationary. The p value is {pvalue} ")

Time series is not stationary. The p value is 0.37677707077291045 


In [ ]:
# If pvalue is not stationary you need to perform differencing
df.y.diff().dropna()

Date
2015-01-02   -138.724112
2015-01-03    172.840266
2015-01-04   -131.864264
2015-01-05    163.120544
2015-01-06   -200.590336
                 ...    
2020-12-27   -227.008591
2020-12-28    312.136144
2020-12-29   -150.927771
2020-12-30     10.397643
2020-12-31    -73.629549
Name: y, Length: 2191, dtype: float64

In [ ]:
# test
pvalue = adfuller(x=df.y.diff().dropna())[1]

In [ ]:
# condition to read test
if pvalue < 0.05:
    print(f"Time series is stationary with a p value of {pvalue}")
else:
    print(f"Time series is not stationary. The p value is {pvalue} ")

Time series is stationary with a p value of 3.3557739456287946e-22


## Sarimax Model

In [ ]:
#Date Handling

model = pm.ARIMA(order=(1, 1, 1),
                 seasonal_order=(1, 1, 1, 7),
                 X=X,
                 suppress_warning=True,
                 force_stationality=False)

In [ ]:
# Cross Validation
cv = model_selection.RollingForecastCV(h=31,
                                        step=16,
                                        initial=df.shape[0] - 180)

In [ ]:
cv_score = model_selection.cross_val_score(model,
                                           y=df.y,
                                           scoring='mean_squared_error',
                                           cv=cv,
                                           verbose=2,
                                           error_score=1000000000000000)

[CV] fold=0 ..........................................................


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


[CV] fold=1 ..........................................................


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


[CV] fold=2 ..........................................................


...

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


[CV] fold=8 ..........................................................


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


[CV] fold=9 ..........................................................


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


In [ ]:
np.sqrt(np.average(cv_score))

59.9552109996143

## Parameter Tuning

In [ ]:
# Grid
param_grid = {'p': [0, 1],
              'd': [1],
              'q': [0, 1],
              'P': [0, 1],
              'D': [0, 1],
              'Q': [0, 1]}

In [ ]:
grid = ParameterGrid(param_grid)
print(len(grid))

32


In [ ]:
# Parameter Tuning
rmse = []
i = 1

In [ ]:
# Parameter Loop
for params in grid:
    print(f"{i} / {len(list(grid))}")
    # model
    model = pm.ARIMA(order=(params['p'], params['d'], params['q']),
                     seasonal_order=(params['P'], params['D'], params['Q'], 7),
                     X=X,
                     suppress_warning=True,
                     force_stationality=False)

    # CV
    cv = model_selection.RollingForecastCV(h=31,
                                            step=16,
                                            initial=df.shape[0] - 180)

    cv_score = model_selection.cross_val_score(model,
                                               y=df.y,
                                               scoring='mean_squared_error',
                                               cv=cv,
                                               verbose=True,
                                               error_score=1000000000000000)
    # Error
    error = np.sqrt(np.mean(cv_score))
    rmse.append(error)

    i += 1

1 / 32


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dat

2 / 32


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dat

3 / 32


...

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dat

31 / 32


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dat

32 / 32


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dat

In [ ]:
# Check the results
tuning_results = pd.DataFrame(grid)
tuning_results['rmse'] = rmse
tuning_results

,D,P,Q,d,p,q,rmse
0,0,0,0,1,0,0,109.196397
1,0,0,0,1,0,1,86.990889
2,0,0,0,1,1,0,88.724335
3,0,0,0,1,1,1,85.350878
4,0,0,1,1,0,0,111.422774
5,0,0,1,1,0,1,84.774305
6,0,0,1,1,1,0,91.853986
7,0,0,1,1,1,1,83.556196
8,0,1,0,1,0,0,134.711122
9,0,1,0,1,0,1,91.895286


In [ ]:
# export best parameters
best_params = tuning_results[tuning_results.rmse == tuning_results.rmse.min()].transpose()
best_params.to_csv("Forecasting Product/best_params_sarimax.csv")